<a href="https://colab.research.google.com/github/mmonch/Sidecar_Project/blob/main/notebooks/Sidecar_Project_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a id='Q0'></a>
<center><a target="_blank" href="https://sit.academy/"><img src="https://drive.google.com/uc?id=1z0U84GYqhbWWpCenFajh8_8XFRGyOc3U" width="200" style="background:none; border:none; box-shadow:none;" /></a> </center>
<center> <h1> Notebook 2: Seq2Seq Model </h1> </center>
<p style="margin-bottom:1cm;"></p>
<center><h4>Marlies Monch, SIT Academy, 2022</h4></center>
<p style="margin-bottom:1cm;"></p>

<div style="background:#EEEDF5;border-top:0.1cm solid #EF475B;border-bottom:0.1cm solid #EF475B;">
    <div style="margin-left: 0.5cm;margin-top: 0.5cm;margin-bottom: 0.5cm;color:#303030">
        <p><strong>Goal:</strong> Run a seq2seq model (RNN) on the attribute technical names to match the attribute business names</p>
        <strong> Outline:</strong>
        <a id='P0' name="P0"></a>
        <ol>
            <li> <a style="color:#303030" href='#I'>Introduction </a> </li>
            <li> <a style="color:#303030" href='#SU'>Set up</a></li>
            <li> <a style="color:#303030" href='#P1'>Part 1</a></li>
            <li> <a style="color:#303030" href='#P2'>Part 2</a></li>
            <li> <a style="color:#303030" href='#P3'>Part 3</a></li>
            <li> <a style="color:#303030" href='#CL'>Conclusion</a></li>
        </ol>
        <strong>Keywords:</strong> data preprocessing, seq2seq, NLP, Sidecar attribute names.
    </div>
</div>
</nav>

<a id='I' name="I"></a>
## [Introduction](#P0)

Sources:

https://keras.io/examples/nlp/neural_machine_translation_with_transformer/

https://loeb.nyc/blog/data-science-word-expander

https://towardsdatascience.com/nlp-building-text-cleanup-and-preprocessing-pipeline-eba4095245a0

https://towardsdatascience.com/guide-to-fine-tuning-text-generation-models-gpt-2-gpt-neo-and-t5-dc5de6b3bc5e

https://www.machinecurve.com/index.php/2020/12/29/differences-between-autoregressive-autoencoding-and-sequence-to-sequence-models-in-machine-learning/

<a id='SU' name="SU"></a>
## [Set up](#P0)

### Magics and Package Installations

In [1]:
!pip install nb_black
!pip install contractions
!pip install textsearch
!pip install tqdm
!pip install --upgrade IPython

     |████████████████████████████████| 156 kB 5.3 MB/s 
     |████████████████████████████████| 843 kB 63.3 MB/s 
  Created wheel for nb-black: filename=nb_black-1.0.7-py3-none-any.whl size=5297 sha256=b2b8245e358d0def1b4deeffa4efe33aae99e2be9be9a342ad9db77a3c08b656
  Stored in directory: /root/.cache/pip/wheels/1e/b2/88/51c66d23ea5fd0d40ed50997555e15d981d92671376a9a412a
Successfully built nb-black
  Attempting uninstall: tomli
    Found existing installation: tomli 2.0.0
    Uninstalling tomli-2.0.0:
      Successfully uninstalled tomli-2.0.0
     |████████████████████████████████| 321 kB 5.2 MB/s 
     |████████████████████████████████| 284 kB 60.5 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85461 sha256=f6fca3d6133f6590cc18ac924683037d42272ac578add7863f9bba339df95ae3
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick
     |█████████████

In [2]:
# auto reload packages and modules when they are modified
%load_ext autoreload
%autoreload 2
# draw matplotlib plots in line
%matplotlib inline
# enforce PEP 8 code on jupyter lab ...
#%load_ext lab_black
# ... or jupyter notebook
%load_ext nb_black

<IPython.core.display.Javascript object>

### Packages

In [36]:
import nltk
import pandas as pd
import numpy as np
import tqdm
import tensorflow as tf
import unicodedata
import re
import contractions
import sklearn
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import TextVectorization

# from nltk get "punkt"
nltk.download('punkt')

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<IPython.core.display.Javascript object>

### Custom classes and functions

### Global Parameters Setting

### User-Dependent Variables

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


<IPython.core.display.Javascript object>

In [5]:
data = pd.read_csv("gdrive/My Drive/SIDECAR_P/Sidecar_Data_Sample.csv")

<IPython.core.display.Javascript object>

<a id='P1'></a>
## [Part one](#P0)

Here you could write a paragraph on the aim of part one. You could also have some nice figures



In [6]:
data.head()

,Domain_Id,Domain_Name,Domain_Desc,Domain_Code,Domain_Status,Domain_Status_Code,Application_Id,Application_Name,Application_Code,Application_Desc,Application_Status,Application_Status_Code,Asset_Id,Asset_Technical_Name,Asset_Business_Name,Asset_Business_Desc,Asset_Status,Asset_Status_Code,Asset_Server_Name,Asset_Database,Asset_Schema,Asset_Type_Id,Asset_Type_Name,Asset_Type_Desc,Privacy_Level_Id,Asset_Privacy_Level,Asset_Privacy_Desc,Attribute_Id,Attribute_Technical_Name,Attribute_Business_Name,Attribute_Business_Desc,Attribute_Business_Rule,Attribute_Data_Type,Attribute_IsPK,Attribute_IsNullable,Attribute_Status,Attribute_Status_Code
0,1,Patient administrative,Patient administrative data (demographic & adm...,PAT_ADM,Active,1,1,Patient Manager,PATMAN,Patient Manager tool is the application allowi...,Active,1,1,PMS_PT,Patient,Patient table Contain personal information abo...,Active,1,SWT01_PRD,PMS,dbo,1,Table,Database table,4,Level 4 - Highly sensitive,Highly sensitive - Internal use,1,ID,Technical Id of the patient,The Primary Key (surrogate unique identifier) ...,NaN,NUMBER (38),Yes,Yes,Active,1
1,1,Patient administrative,Patient administrative data (demographic & adm...,PAT_ADM,Active,1,1,Patient Manager,PATMAN,Patient Manager tool is the application allowi...,Active,1,1,PMS_PT,Patient,Patient table Contain personal information abo...,Active,1,SWT01_PRD,PMS,dbo,1,Table,Database table,4,Level 4 - Highly sensitive,Highly sensitive - Internal use,2,GNDR_CD,Gender Code,Code of GENDER CODE defined as: The gender of ...,"If Gender code = M then Male, F = Female, or U...",VARCHAR2 (80),No,No,Active,1
2,1,Patient administrative,Patient administrative data (demographic & adm...,PAT_ADM,Active,1,1,Patient Manager,PATMAN,Patient Manager tool is the application allowi...,Active,1,1,PMS_PT,Patient,Patient table Contain personal information abo...,Active,1,SWT01_PRD,PMS,dbo,1,Table,Database table,4,Level 4 - Highly sensitive,Highly sensitive - Internal use,3,LIVG_ARNGMNT_CD,Living arrangement,Code of LIVING ARRANGEMENT CODE defined as: A ...,NaN,VARCHAR2 (80),No,No,Active,1
3,1,Patient administrative,Patient administrative data (demographic & adm...,PAT_ADM,Active,1,1,Patient Manager,PATMAN,Patient Manager tool is the application allowi...,Active,1,1,PMS_PT,Patient,Patient table Contain personal information abo...,Active,1,SWT01_PRD,PMS,dbo,1,Table,Database table,4,Level 4 - Highly sensitive,Highly sensitive - Internal use,4,MRTL_STUS_CD,Marital status code,Code of MARITAL STATUS CODE defined as: Party'...,NaN,VARCHAR2 (80),No,No,Active,1
4,1,Patient administrative,Patient administrative data (demographic & adm...,PAT_ADM,Active,1,1,Patient Manager,PATMAN,Patient Manager tool is the application allowi...,Active,1,1,PMS_PT,Patient,Patient table Contain personal information abo...,Active,1,SWT01_PRD,PMS,dbo,1,Table,Database table,4,Level 4 - Highly sensitive,Highly sensitive - Internal use,5,OCUPATN_CD,Occupation code,Code of OCCUPATION CODE defined as: The occupa...,NaN,VARCHAR2 (80),No,No,Active,1


<IPython.core.display.Javascript object>

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.isnull().any()

In [ ]:
is_na = data[data.isna().any(axis=1)]

In [ ]:
len(is_na)

In [ ]:
is_na.tail()

In [ ]:
np.sort(data["Asset_Id"].unique())

In [ ]:
# how many values per column, value_counts() per category
for col in data.columns:
  print (f"Column Name, Number of unique values: \n{col}, {data[col].nunique()}\nValue Counts: \n{data[col].value_counts()}\n\n----------------------")

### Exploring Basic Models

First we will try to generate the Business Name from the technical name, which should be a relatively straightforward problem. 

There are two basic approaches we can try. 

1. Similarity-based Model
2. Rule-based Model

Caveats:
- Where preprocessing already has generated the business label
- unbalanced data set
- small data set 


In [7]:
# preprocess and normalize Text

# in case text not english
def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

# preprocessing steps
  # remove excess white spaces
  # remove underscores
  # tokenize words
def pre_process_corpus(docs):
  norm_docs = []
  for string in tqdm.tqdm(docs):
    string = string.replace("_", " ")
    string = string.translate(string.maketrans("\n\t\r", "   "))
    string = string.lower()
    string = remove_accented_chars(string)
    string = contractions.fix(string)
    # lower case and remove special characters or whitespaces
    string = re.sub(r'[^a-zA-Z0-9\s]', '', string, flags=re.I|re.A)
    string = string.strip()
    norm_docs.append(string)

  return norm_docs

<IPython.core.display.Javascript object>

In [8]:
prep_tech = pre_process_corpus(data["Attribute_Technical_Name"])


100%|██████████| 4133/4133 [00:00<00:00, 85029.55it/s]


<IPython.core.display.Javascript object>

In [9]:
prep_business = pre_process_corpus(data["Attribute_Business_Name"])

100%|██████████| 4133/4133 [00:00<00:00, 81617.46it/s]


<IPython.core.display.Javascript object>

In [10]:
# parse text into text pairs
tech = pd.DataFrame(prep_tech, columns =["tech_name"], dtype="string")
busi = pd.DataFrame(prep_business, columns =["busi_name"], dtype="string")
attribute_df = pd.concat([tech,busi], axis=1)
attribute_df

,tech_name,busi_name
0,id,technical id of the patient
1,gndr cd,gender code
2,livg arngmnt cd,living arrangement
3,mrtl stus cd,marital status code
4,ocupatn cd,occupation code
...,...,...
4128,config assetlist data source,config assetlist data source
4129,attribute sample data,attribute sample data
4130,property isdq,property isdq
4131,property dq calculation,property dq calculation


<IPython.core.display.Javascript object>

In [12]:
# shuffle and define labels
attribute_df = attribute_df.sample(frac=1, random_state=42)
# X = attribute_df["tech_name"]
# Y = attribute_df["busi_name"]

# train test split
# x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, train_size=None, random_state=42)

num_val_samples = int(0.15 * len(attribute_df))
num_train_samples = len(attribute_df) - 2 * num_val_samples
train_pairs = attribute_df[:num_train_samples]
val_pairs = attribute_df[num_train_samples : num_train_samples + num_val_samples]
test_pairs = attribute_df[num_train_samples + num_val_samples :]

print(f"{len(attribute_df)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

4133 total pairs
2895 training pairs
619 validation pairs
619 test pairs


<IPython.core.display.Javascript object>

In [ ]:
# configureations
# batch_size
# training epochs
# latent diemsionality of the encoding space
# number of samplesto train on
# data path

In [14]:
# Vectorize the data

vocab_size = 15000
sequence_length = 20
batch_size = 64

tech_vectorization = TextVectorization(
    max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length,
)
busi_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
)
train_tech_texts = [pair[0] for pair in train_pairs]
train_busi_texts = [pair[1] for pair in train_pairs]
tech_vectorization.adapt(train_tech_texts)
busi_vectorization.adapt(train_busi_texts)

<IPython.core.display.Javascript object>

In [23]:
train_pairs = train_pairs.to_records(index=False)
train_pairs = list(train_pairs)


AttributeError: ignored

<IPython.core.display.Javascript object>

In [24]:
val_pairs = val_pairs.to_records(index=False)
val_pairs = list(val_pairs)

<IPython.core.display.Javascript object>

In [ ]:
val_pairs

In [30]:
def format_dataset(tech, busi):
    tech = tech_vectorization(tech)
    busi = busi_vectorization(busi)
    return ({"encoder_inputs": tech, "decoder_inputs": busi[:, :-1],}, busi[:, 1:])


def make_dataset(pairs):
    tech_texts, busi_texts = zip(*pairs)
    tech_texts = list(tech_texts)
    busi_texts = list(busi_texts)
    dataset = tf.data.Dataset.from_tensor_slices((tech_texts, busi_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

<IPython.core.display.Javascript object>

In [31]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


<IPython.core.display.Javascript object>

In [34]:
# Build the model
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

<IPython.core.display.Javascript object>

In [37]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

<IPython.core.display.Javascript object>

In [40]:
# Compile the model
epochs = 30  # This should be at least 30 for convergence

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
# train the model with fit
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)
# save model
transformer.save("transformer_RNN_attribute_labels")

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding (Position  (None, None, 256)   3845120     ['encoder_inputs[0][0]']         
 alEmbedding)                                                                                     
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder (Transform  (None, None, 256)   3155456     ['positional_embedding[

INFO:tensorflow:Assets written to: transformer_RNN_attribute_labels/assets


INFO:tensorflow:Assets written to: transformer_RNN_attribute_labels/assets
/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


<IPython.core.display.Javascript object>

In [ ]:
# train the model with fit
# save model

<div style="background:#EEEDF5;color:#303030"><div style="margin-left: .2cm">
    <b>Comments:</b>
</div></div>

<div style="margin-left: 0.4cm">
    This Markdown cell is a template to write extended comments. e.g. explain the approach taken, or concepts behind algorithm chosen. Aim at having 2 lines at least. Anything smaller can be directly integrated in the code, or gathered together with other comments to form a bigger paragraph. Since this block is within a div element, you need to use html formatting syntax, not markdown
</div>

<div style="background:#EEEDF5;border:0.1cm solid #FDB100;color:#303030">
    <div style="margin: 0.2cm 0.2cm 0.2cm 0.2cm">
        <b style="color:#FDB100">Title:</b> 
        This is a an example yellow box
    </div>
</div>

<div style="background:#EEEDF5;border:0.1cm solid #00BAE5;color:#303030">
    <div style="margin: 0.2cm 0.2cm 0.2cm 0.2cm">
        <b style="color:#00BAE5">Note:</b>
        This is an example Blue Box
    </div>
</div>

<div style="background:#EEEDF5;border:0.1cm solid #EF475B;color:#303030">
    <div style="margin: 0.2cm 0.2cm 0.2cm 0.2cm">
        <b style="color:#EF475B">Warning:</b> 
        This is an example red box
    </div>
</div>

<a id='P2' name="P2"></a>
## [Part 2 title here](#P0)

<a id='P3' name="P3"></a>
## [Part 3 title here](#P0)

<a id='CL'></a>
## [Conclusion](#P0)

This is a pre-written conclusion in which we have nice figures.

<div style="border-top:0.1cm solid #EF475B"></div>
    <strong><a href='#Q0'><div style="text-align: right"> <h3>End of this Notebook.</h3></div></a></strong>